In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re

class DataCleaningSystem:
    def __init__(self):
        self.df = None
        self.report = {}

    def load_file(self, filepath, filetype="csv", db_conn=None, query=None):
        if filetype == "csv":
            self.df = pd.read_csv(filepath)
        elif filetype == "excel":
            self.df = pd.read_excel(filepath)
        elif filetype == "json":
            self.df = pd.read_json(filepath)
        elif filetype == "sql" and db_conn and query:
            engine = create_engine(db_conn)
            self.df = pd.read_sql(query, engine)
        else:
            raise ValueError("Unsupported file type or missing parameters.")
        print("\n✅ File loaded successfully!")
        return self.df.head()

    def remove_duplicates(self):
        before = len(self.df)
        self.df.drop_duplicates(inplace=True)
        after = len(self.df)
        self.report["duplicates_removed"] = before - after
        print(f"✅ Removed {before - after} duplicates.")

    def handle_missing(self, method="fill", value="Unknown"):
        before = self.df.isnull().sum().sum()
        if method == "drop":
            self.df.dropna(inplace=True)
        elif method == "fill":
            self.df.fillna(value, inplace=True)
        after = self.df.isnull().sum().sum()
        self.report["missing_handled"] = before - after
        print(f"✅ Handled {before - after} missing values.")

    def standardize_emails(self):
        if "email" in self.df.columns:
            self.df["email"] = self.df["email"].astype(str).str.lower().str.strip()
            print("✅ Emails standardized.")

    def clean_phone_numbers(self):
        if "phone" in self.df.columns:
            self.df["phone"] = self.df["phone"].astype(str).apply(lambda x: re.sub(r'\D', '', x))
            print("✅ Phone numbers cleaned.")

    def normalize_columns(self):
        self.df.columns = [col.strip().lower().replace(" ", "_") for col in self.df.columns]
        print("✅ Column names normalized.")

    def convert_dates(self, colname):
        if colname in self.df.columns:
            self.df[colname] = pd.to_datetime(self.df[colname], errors="coerce").dt.strftime("%Y-%m-%d")
            print(f"✅ Converted {colname} to YYYY-MM-DD format.")

    def export_file(self, filename, filetype="csv"):
        if filetype == "csv":
            self.df.to_csv(filename, index=False)
        elif filetype == "excel":
            self.df.to_excel(filename, index=False)
        elif filetype == "json":
            self.df.to_json(filename, orient="records")
        else:
            raise ValueError("Unsupported export format.")
        print(f"✅ File exported as {filename}")

    def generate_report(self):
        print("\n📊 Cleaning Report:")
        for k, v in self.report.items():
            print(f"- {k}: {v}")


def main_menu(system):
    while True:
        print("\n==== MAIN MENU ====")
        print("1. Load File")
        print("2. Preview Data")
        print("3. Cleaning Options")
        print("4. Export Cleaned Data")
        print("5. Generate Report")
        print("6. Exit")

        choice = input("Enter choice: ")

        if choice == "1":
            filepath = input("Enter file path: ")
            ftype = input("Enter file type (csv/excel/json): ")
            system.load_file(filepath, ftype)

        elif choice == "2":
            if system.df is not None:
                print(system.df.head())
            else:
                print("⚠️ No data loaded.")

        elif choice == "3":
            cleaning_menu(system)

        elif choice == "4":
            filename = input("Enter filename to save: ")
            ftype = input("Enter file type (csv/excel/json): ")
            system.export_file(filename, ftype)

        elif choice == "5":
            system.generate_report()

        elif choice == "6":
            print("👋 Exiting system.")
            break
        else:
            print("❌ Invalid choice.")


def cleaning_menu(system):
    while True:
        print("\n---- CLEANING MENU ----")
        print("1. Remove Duplicates")
        print("2. Handle Missing Values")
        print("3. Standardize Emails")
        print("4. Clean Phone Numbers")
        print("5. Normalize Column Names")
        print("6. Convert Date Column")
        print("7. Back to Main Menu")

        choice = input("Enter choice: ")

        if choice == "1":
            system.remove_duplicates()
        elif choice == "2":
            method = input("Choose method (drop/fill): ")
            system.handle_missing(method)
        elif choice == "3":
            system.standardize_emails()
        elif choice == "4":
            system.clean_phone_numbers()
        elif choice == "5":
            system.normalize_columns()
        elif choice == "6":
            col = input("Enter column name to convert: ")
            system.convert_dates(col)
        elif choice == "7":
            break
        else:
            print("❌ Invalid choice.")


# Initialize system
system = DataCleaningSystem()
main_menu(system)